# MMDetection Test and Evaluation Script
이 노트북은 MMDetection을 사용하여 모델을 테스트하고 평가하는 기능을 제공합니다. 테스트 데이터셋 경로 및 기타 파라미터를 쉽게 수정할 수 있습니다.

## Step 1: Set Paths
테스트에 사용할 설정 파일, 체크포인트 파일, 테스트 데이터셋의 경로를 지정하세요.

In [1]:
import os
import os.path as osp
import warnings
import json
from copy import deepcopy
import pandas as pd
from pycocotools.coco import COCO

from mmengine import ConfigDict
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.evaluation import DumpDetResults
from mmdet.utils import setup_cache_size_limit_of_dynamo


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def increment_category_id(annotation_file, output_file):
    """
    COCO 형식의 어노테이션 파일에서 'category_id'를 1부터 시작하도록 수정합니다.
    
    Args:
        annotation_file (str): 원본 어노테이션 파일 경로.
        output_file (str): 수정된 어노테이션 파일을 저장할 경로.
    """
    with open(annotation_file, 'r') as f:
        data = json.load(f)
    
    # 'categories' 섹션의 'id'를 1부터 시작하도록 수정
    id_mapping = {}
    for category in data['categories']:
        old_id = category['id']
        new_id = old_id + 1
        id_mapping[old_id] = new_id
        category['id'] = new_id
    
    # 'annotations' 섹션의 'category_id'를 매핑에 따라 수정
    for ann in data['annotations']:
        old_cat_id = ann['category_id']
        if old_cat_id in id_mapping:
            ann['category_id'] = id_mapping[old_cat_id]
        else:
            print(f"Warning: annotation id {ann['id']} has invalid category_id {old_cat_id}")
    
    # 이미지의 'id'는 변경하지 않음
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)


# 어노테이션 파일 경로 설정
train_ann_file = '/data/ephemeral/home/dataset/train_split.json'
val_ann_file = '/data/ephemeral/home/dataset/val_split.json'
test_ann_file = '/data/ephemeral/home/dataset/test.json'

# 수정된 어노테이션 파일 저장 경로
train_fixed_ann_file = '/data/ephemeral/home/dataset/train_split_fixed.json'
val_fixed_ann_file = '/data/ephemeral/home/dataset/val_split_fixed.json'
test_fixed_ann_file = '/data/ephemeral/home/dataset/test_fixed.json'

# 어노테이션 파일에 category_id를 1부터 시작하도록 수정
increment_category_id(train_ann_file, train_fixed_ann_file)
increment_category_id(val_ann_file, val_fixed_ann_file)
increment_category_id(test_ann_file, test_fixed_ann_file)

print("어노테이션 파일 수정 완료: train_fixed.json, val_fixed.json, test_fixed.json")

어노테이션 파일 수정 완료: train_fixed.json, val_fixed.json, test_fixed.json


In [3]:
# 테스트 어노테이션 파일 로드 및 정보 출력
with open(test_fixed_ann_file, 'r') as f:
    data = json.load(f)

print("Number of images:", len(data['images']))
print("Categories:", data['categories'][:5])  # 상위 5개 카테고리만 출력

# 각 이미지의 id와 file_name 출력
for img in data['images']:
    print(f"Image ID: {img['id']}, File Name: {img['file_name']}")

Number of images: 4871
Categories: [{'id': 1, 'name': 'General trash', 'supercategory': 'General trash'}, {'id': 2, 'name': 'Paper', 'supercategory': 'Paper'}, {'id': 3, 'name': 'Paper pack', 'supercategory': 'Paper pack'}, {'id': 4, 'name': 'Metal', 'supercategory': 'Metal'}, {'id': 5, 'name': 'Glass', 'supercategory': 'Glass'}]
Image ID: 0, File Name: test/0000.jpg
Image ID: 1, File Name: test/0001.jpg
Image ID: 2, File Name: test/0002.jpg
Image ID: 3, File Name: test/0003.jpg
Image ID: 4, File Name: test/0004.jpg
Image ID: 5, File Name: test/0005.jpg
Image ID: 6, File Name: test/0006.jpg
Image ID: 7, File Name: test/0007.jpg
Image ID: 8, File Name: test/0008.jpg
Image ID: 9, File Name: test/0009.jpg
Image ID: 10, File Name: test/0010.jpg
Image ID: 11, File Name: test/0011.jpg
Image ID: 12, File Name: test/0012.jpg
Image ID: 13, File Name: test/0013.jpg
Image ID: 14, File Name: test/0014.jpg
Image ID: 15, File Name: test/0015.jpg
Image ID: 16, File Name: test/0016.jpg
Image ID: 17, F

In [4]:
# 설정 파일 경로 및 체크포인트 경로 설정
config_path = './projects/CO-DETR/configs/codino/co_dino_5scale_swin_l_16xb1_1x_coco.py'  # 설정 파일 경로
checkpoint_path = './work_dirs/co_dino_custom/epoch_24.pth'  # 체크포인트 파일 경로
work_dir = './work_dirs/co_dino_custom'  # 작업 디렉토리

# 테스트 데이터셋 경로 설정
test_data_prefix = '/data/ephemeral/home/dataset/test/'  # 테스트 이미지 디렉토리 경로

# 클래스 정의 (학습과 동일하게 설정)
classes = (
    "General trash", "Paper", "Paper pack", "Metal", "Glass",
    "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"
)
num_classes = len(classes)  # 10

## Step 2: Load Configuration
설정 파일을 로드하고 테스트 데이터셋 경로를 업데이트합니다.

In [5]:
# 설정 파일 로드
cfg = Config.fromfile(config_path)
cfg.launcher = 'none'
cfg.work_dir = work_dir
cfg.load_from = checkpoint_path

# 데이터셋 루트 경로 수정
data_root = '/data/ephemeral/home/dataset/'

# 테스트 데이터셋 설정 업데이트
cfg.test_dataloader.dataset.type = 'CocoDataset'
cfg.test_dataloader.dataset.data_root = data_root
cfg.test_dataloader.dataset.ann_file = 'test_fixed.json'  # 수정된 파일 사용
cfg.test_dataloader.dataset.data_prefix = dict(img='')  # 'test/' 제거
cfg.test_dataloader.dataset.test_mode = True

# 클래스 설정을 데이터셋의 metainfo에 추가
cfg.test_dataloader.dataset.metainfo = dict(classes=classes)  # 테스트 데이터셋 클래스 설정

In [6]:
# 테스트 파이프라인에서 LoadAnnotations 제거 (테스트 데이터에 어노테이션이 없을 경우)
test_pipeline = deepcopy(cfg.test_dataloader.dataset.pipeline)
test_pipeline = [step for step in test_pipeline if step['type'] != 'LoadAnnotations']
cfg.test_dataloader.dataset.pipeline = test_pipeline


## Step 3: Configure Test-Time Augmentation (Optional)
테스트 시 Test-Time Augmentation(TTA)을 적용하고 싶다면 여기서 활성화하세요.

In [7]:
# TTA 사용 여부 설정 (False로 설정)
use_tta = False

if use_tta:
    if 'tta_model' not in cfg:
        warnings.warn('설정 파일에서 `tta_model`을 찾을 수 없습니다. 기본 설정으로 대체합니다.')
        cfg.tta_model = dict(
            type='DetTTAModel',
            tta_cfg=dict(nms=dict(type='nms', iou_threshold=0.5), max_per_img=100))
    if 'tta_pipeline' not in cfg:
        warnings.warn('설정 파일에서 `tta_pipeline`을 찾을 수 없습니다. 기본 설정으로 대체합니다.')
        test_data_cfg = cfg.test_dataloader.dataset
        while 'dataset' in test_data_cfg:
            test_data_cfg = test_data_cfg['dataset']
        cfg.tta_pipeline = deepcopy(test_data_cfg.pipeline)
        flip_tta = dict(
            type='TestTimeAug',
            transforms=[
                [
                    dict(type='RandomFlip', prob=1.),
                    dict(type='RandomFlip', prob=0.)
                ],
                [
                    dict(
                        type='PackDetInputs',
                        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor', 'flip', 'flip_direction')
                    )
                ]
            ]
        )
        cfg.tta_pipeline[-1] = flip_tta
    cfg.model = ConfigDict(**cfg.tta_model, module=cfg.model)
    cfg.test_dataloader.dataset.pipeline = cfg.tta_pipeline

In [8]:
# 테스트 평가기 업데이트: CocoMetric 사용
cfg.test_evaluator = dict(
    type='CocoMetric',
    ann_file=test_fixed_ann_file,  # 테스트 어노테이션 파일 경로
    metric=['bbox'],  # 평가 지표 설정 ('bbox'를 포함하여야 self.cat_ids가 설정됨)
    format_only=True,  # 결과만 포맷
    outfile_prefix=osp.join(work_dir, 'test_results'),  # 결과 파일 저장 경로 (test_results.json)
    backend_args=None
)


## Step 4: Build Runner and Perform Testing
설정된 설정 파일과 체크포인트를 사용하여 러너를 생성하고 테스트를 수행합니다.

In [9]:
# 러너 생성
runner = Runner.from_cfg(cfg)

# 테스트 수행
output = runner.test()

10/17 14:10:54 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1764336402
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,cod

/data/ephemeral/home/baseline/mmdetection/projects/CO-DETR/codetr/transformer.py:1325: UserWarning: If you want to reduce GPU memory usage,                               please install fairscale by executing the                               following command: pip install fairscale.
  warnings.warn('If you want to reduce GPU memory usage, \
/data/ephemeral/home/baseline/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


10/17 14:11:02 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/17 14:11:02 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/data/ephemeral/home/baseline/mmdetection/projects/CO-DETR/codetr/transformer.py:991: UserWarning: __floordiv__ is deprecated, and its behavior will change i

10/17 14:11:22 - mmengine - INFO - Epoch(test) [  50/4871]    eta: 0:24:09  time: 0.3006  data_time: 0.0034  memory: 1749  
10/17 14:11:34 - mmengine - INFO - Epoch(test) [ 100/4871]    eta: 0:21:55  time: 0.2508  data_time: 0.0015  memory: 1749  
10/17 14:11:47 - mmengine - INFO - Epoch(test) [ 150/4871]    eta: 0:21:02  time: 0.2509  data_time: 0.0019  memory: 1749  
10/17 14:12:00 - mmengine - INFO - Epoch(test) [ 200/4871]    eta: 0:20:29  time: 0.2509  data_time: 0.0016  memory: 1749  
10/17 14:12:12 - mmengine - INFO - Epoch(test) [ 250/4871]    eta: 0:20:04  time: 0.2503  data_time: 0.0016  memory: 1749  
10/17 14:12:25 - mmengine - INFO - Epoch(test) [ 300/4871]    eta: 0:19:43  time: 0.2506  data_time: 0.0016  memory: 1749  
10/17 14:12:37 - mmengine - INFO - Epoch(test) [ 350/4871]    eta: 0:19:25  time: 0.2511  data_time: 0.0017  memory: 1749  
10/17 14:12:50 - mmengine - INFO - Epoch(test) [ 400/4871]    eta: 0:19:09  time: 0.2513  data_time: 0.0016  memory: 1749  
10/17 14

## Step 5: Create Submission File
요구되는 제출 형식에 맞는 CSV 파일을 생성합니다.

In [17]:
# COCO 파일을 로드하여 이미지 정보 가져오기
coco = COCO(test_fixed_ann_file)
img_ids = coco.getImgIds()
img_info = coco.loadImgs(img_ids)

# 이미지 ID를 file_name으로 매핑
id_to_filename = {img['id']: img['file_name'] for img in img_info}

# 테스트 결과 로드
test_results_file = osp.join(work_dir, 'test_results.bbox.json')
with open(test_results_file, 'r') as f:
    results = json.load(f)

# 이미지별로 예측 결과를 그룹화
from collections import defaultdict

imgid_to_results = defaultdict(list)
for res in results:
    imgid_to_results[res['image_id']].append(res)

prediction_strings = []
file_names = []

for img_id in img_ids:
    preds = imgid_to_results.get(img_id, [])
    prediction_string = ''
    for pred in preds:
        cat_id_original = pred['category_id']
        score = pred['score']
        
        # 'General trash' 클래스에 대해서만 confidence score 필터링 적용
        if cat_id_original == 8 and score < 0.3:
            continue  # 'General trash' 클래스이고, 신뢰 점수가 0.3 미만인 경우 제외
        
        cat_id = cat_id_original - 1  # 원래 category_id를 사용 (0-9)
        bbox = pred['bbox']  # [x, y, width, height]
        # COCO 형식의 bbox는 [x, y, width, height], 이를 [x1, y1, x2, y2]로 변환
        x1 = bbox[0]
        y1 = bbox[1]
        x2 = x1 + bbox[2]
        y2 = y1 + bbox[3]
        prediction_string += f"{cat_id} {score:.8f} {x1:.5f} {y1:.5f} {x2:.5f} {y2:.5f} "
    prediction_strings.append(prediction_string.strip())
    file_names.append(id_to_filename[img_id])

# 제출용 데이터프레임 생성
submission = pd.DataFrame({
    'PredictionString': prediction_strings,
    'image_id': file_names
})

# 제출 CSV 파일 저장
submission_file = osp.join(work_dir, 'submission.csv')
submission.to_csv(submission_file, index=False)
print(f"제출 파일이 저장되었습니다: {submission_file}")

# 제출 파일의 첫 몇 개 항목 표시
print(submission.head())

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
제출 파일이 저장되었습니다: ./work_dirs/co_dino_custom/submission.csv
                                    PredictionString       image_id
0  7 0.90593171 215.00116 51.44936 454.71326 472....  test/0000.jpg
1  4 0.90134412 343.03821 249.40767 753.09290 696...  test/0001.jpg
2  1 0.82718897 308.63757 319.61102 1024.00000 74...  test/0002.jpg
3  9 0.93017519 142.90448 264.32623 915.63599 824...  test/0003.jpg
4  0 0.76909906 426.83032 408.41565 658.07764 574...  test/0004.jpg


In [18]:
# 제출 파일의 첫 몇 개 항목 표시
print(submission.head(50))

                                     PredictionString       image_id
0   7 0.90593171 215.00116 51.44936 454.71326 472....  test/0000.jpg
1   4 0.90134412 343.03821 249.40767 753.09290 696...  test/0001.jpg
2   1 0.82718897 308.63757 319.61102 1024.00000 74...  test/0002.jpg
3   9 0.93017519 142.90448 264.32623 915.63599 824...  test/0003.jpg
4   0 0.76909906 426.83032 408.41565 658.07764 574...  test/0004.jpg
5   2 0.93671900 353.14499 225.30913 778.56726 713...  test/0005.jpg
6   4 0.66947091 361.90643 451.83389 846.58362 725...  test/0006.jpg
7   1 0.57664686 885.40503 895.38123 1023.54633 10...  test/0007.jpg
8   7 0.82741451 164.31213 531.74280 468.19257 965...  test/0008.jpg
9   0 0.91106886 290.23840 114.42679 662.62000 980...  test/0009.jpg
10  4 0.90811294 201.66687 320.88403 653.02295 481...  test/0010.jpg
11  6 0.75647712 0.00000 158.01788 539.25415 778.0...  test/0011.jpg
12  7 0.89079052 291.00201 407.54138 551.31451 790...  test/0012.jpg
13  1 0.73638386 288.69244 308.709

In [19]:
# 제출 파일 로드 후 'General trash' 클래스의 confidence score 확인
loaded_submission = pd.read_csv(submission_file)
for idx, row in loaded_submission.iterrows():
    preds = row['PredictionString'].split(' ')
    # 각 예측은 6개의 값으로 구성되어 있으므로 6개씩 분할
    for i in range(0, len(preds), 6):
        if i+1 >= len(preds):
            break
        cat_id = int(preds[i])
        score = float(preds[i+1])
        if cat_id == 7 and score < 0.3:
            raise AssertionError(f"Image {row['image_id']} has 'General trash' prediction with score {score} < 0.3")
print("모든 'General trash' 예측의 점수가 0.3 이상입니다.")


모든 'General trash' 예측의 점수가 0.3 이상입니다.
